In [1]:
import os
from PIL import Image
import numpy as np
import cv2
import tensorflow as tf
from concurrent.futures import ProcessPoolExecutor


2025-02-05 14:57:14.303643: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-05 14:57:14.450474: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Generate

# For Classifiaction (Low,High)

In [56]:
PATH_TO_TILES = "Tiles"
PATH_TO_COORDS = "Coordinates"
GENERATED_DIR = "Generated"
IMAGE_SIZE_W = 256
IMAGE_SIZE_L = 256

os.makedirs(GENERATED_DIR, exist_ok=True)

for subfolder in os.listdir(PATH_TO_COORDS):
    coords_subfolder_path = os.path.join(PATH_TO_COORDS, subfolder)
    tiles_subfolder_path = os.path.join(PATH_TO_TILES, subfolder)
    generated_subfolder_path = os.path.join(GENERATED_DIR, subfolder)

    if not os.path.exists(tiles_subfolder_path):
        continue

    os.makedirs(generated_subfolder_path, exist_ok=True)

    for coord_file in os.listdir(coords_subfolder_path):
        if coord_file.endswith("_coords.txt"):
            base_name = coord_file[:-11]

            coord_file_path = os.path.join(coords_subfolder_path, coord_file)
            tile_image_path = os.path.join(tiles_subfolder_path, base_name + ".png")

            if not os.path.exists(tile_image_path):
                continue

            with open(coord_file_path, 'r') as file:
                line_count = file.read().count('\n')

            label = "low" if line_count < 5 else "high"

            with Image.open(tile_image_path) as img:
                resized_img = img.resize((IMAGE_SIZE_W, IMAGE_SIZE_L))
                output_file_name = f"{base_name}_{label}.png"
                output_file_path = os.path.join(generated_subfolder_path, output_file_name)
                resized_img.save(output_file_path)

# For Regression (Msaked based)

In [2]:
PATH_TO_TILES = os.path.expanduser('~/Documents/Code/Lung_Injury/Tiles/')
PATH_TO_COORDS = os.path.expanduser('~/Documents/Code/Lung_Injury/Coordinates/')
GENERATED_DIR = os.path.expanduser('~/Documents/Code/Lung_Injury/Generated_Masks')
RESIZED_TILES_DIR = os.path.expanduser('~/Documents/Code/Lung_Injury/Resized_Tiles')

IMAGE_SIZE_W = 512
IMAGE_SIZE_L = 512

os.makedirs(GENERATED_DIR, exist_ok=True)
os.makedirs(RESIZED_TILES_DIR, exist_ok=True)

def process_file(coord_file_path, tile_image_path, resized_subfolder_path, generated_subfolder_path):
    if os.path.getsize(coord_file_path) == 0:
        print(f"Skipping {coord_file_path} because it is empty.")
        return
    
    base_name = os.path.basename(coord_file_path)[:-11]
    with Image.open(tile_image_path) as img:
        orig_w, orig_h = img.size
        mask = np.zeros((orig_h, orig_w), dtype=np.uint8)
        with open(coord_file_path, "r") as file:
            lines = file.readlines()
            line_count = len(lines)
            for line in lines:
                coords = line.strip().split(",")
                if len(coords) >= 4:
                    x1, y1, x2, y2 = map(int, coords[:4])
                    mask[y1:y2, x1:x2] = 255
        mask_resized = Image.fromarray(mask).resize((IMAGE_SIZE_W, IMAGE_SIZE_L), Image.NEAREST)
        img_resized = img.resize((IMAGE_SIZE_W, IMAGE_SIZE_L))
        resized_tile_path = os.path.join(resized_subfolder_path, f"{base_name}_{line_count}.png")
        img_resized.save(resized_tile_path)
        output_file_path = os.path.join(generated_subfolder_path, f"{base_name}_mask_{line_count}.png")
        mask_resized.save(output_file_path)

def process_subfolder(subfolder):
    coords_subfolder_path = os.path.join(PATH_TO_COORDS, subfolder)
    tiles_subfolder_path = os.path.join(PATH_TO_TILES, subfolder)
    generated_subfolder_path = os.path.join(GENERATED_DIR, subfolder)
    resized_subfolder_path = os.path.join(RESIZED_TILES_DIR, subfolder)
    
    if not os.path.exists(coords_subfolder_path):
        print(f"Skipping {subfolder} because no corresponding coordinates folder exists.")
        return
    
    if not os.path.exists(tiles_subfolder_path):
        print(f"Skipping {subfolder} because no corresponding tiles folder exists.")
        return
    
    os.makedirs(generated_subfolder_path, exist_ok=True)
    os.makedirs(resized_subfolder_path, exist_ok=True)
    
    with ProcessPoolExecutor() as executor:
        futures = []
        for coord_file in os.listdir(coords_subfolder_path):
            if coord_file.endswith("_coords.txt"):
                coord_file_path = os.path.join(coords_subfolder_path, coord_file)
                base_name = coord_file[:-11]
                tile_image_path = os.path.join(tiles_subfolder_path, base_name + ".png")
                if not os.path.exists(tile_image_path):
                    print(f"Skipping {tile_image_path} because the corresponding tile image does not exist.")
                    continue
                futures.append(executor.submit(process_file, coord_file_path, tile_image_path, resized_subfolder_path, generated_subfolder_path))
        for future in futures:
            future.result()

for observer_folder in os.listdir(PATH_TO_TILES):
    observer_path = os.path.join(PATH_TO_TILES, observer_folder)
    if os.path.isdir(observer_path):
        for subfolder in os.listdir(observer_path):
            subfolder_path = os.path.join(observer_path, subfolder)
            if os.path.isdir(subfolder_path):
                process_subfolder(os.path.join(observer_folder, subfolder))

Skipping /home/gandalf/Documents/Code/Lung_Injury/Coordinates/Observer-2 (Prarthna)/ALI_surgical_w_catheter_m_3/tile_65480_54972_coords.txt because it is empty.
Skipping /home/gandalf/Documents/Code/Lung_Injury/Coordinates/Observer-2 (Prarthna)/ALI_surgical_w_catheter_m_3/tile_45836_121142_coords.txt because it is empty.
Skipping /home/gandalf/Documents/Code/Lung_Injury/Coordinates/Observer-2 (Prarthna)/ALI_surgical_w_catheter_m_3/tile_60569_122160_coords.txt because it is empty.
Skipping /home/gandalf/Documents/Code/Lung_Injury/Coordinates/Observer-2 (Prarthna)/ALI_surgical_w_catheter_m_3/tile_42562_119106_coords.txt because it is empty.
Skipping /home/gandalf/Documents/Code/Lung_Injury/Coordinates/Observer-2 (Prarthna)/ALI_surgical_w_catheter_m_3/tile_27829_47846_coords.txt because it is empty.
Skipping /home/gandalf/Documents/Code/Lung_Injury/Coordinates/Observer-2 (Prarthna)/ALI_surgical_w_catheter_m_3/tile_19644_40720_coords.txt because it is empty.
Skipping /home/gandalf/Document

# Loading

# For Classifiaction (Low,High)

In [64]:
image_data = np.empty((0, 256, 256, 3), dtype=np.float32) 
labels = np.empty((0,), dtype=np.int32)

def label_to_numeric(label):
    return 0 if label == "low" else 1  # Map "low" to 0 and "high" to 1

for subfolder in os.listdir(GENERATED_DIR):
    subfolder_path = os.path.join(GENERATED_DIR, subfolder)

    if os.path.isdir(subfolder_path): 
        for file_name in os.listdir(subfolder_path):
            file_path = os.path.join(subfolder_path, file_name)

            if file_name.endswith(".png"):
                label = file_name.split("_")[-1].split(".")[0]  # Get "low" or "high"

                with Image.open(file_path) as img:
                    if img.mode == "RGBA":
                        img = img.convert("RGB")

                    img_array = np.array(img) / 255.0  

                    if img_array.shape != (256, 256, 3):
                        print(f"Skipping image due to unexpected shape: {img_array.shape}")
                        continue

                    image_data = np.vstack([image_data, img_array[np.newaxis, ...]])
                    labels = np.append(labels, label_to_numeric(label))



# Tensorflow

In [74]:
import tensorflow as tf

dataset = tf.data.Dataset.from_tensor_slices((image_data, labels))

dataset = dataset.shuffle(len(image_data)).batch(32)


# For Regression (Msaked based)

## Tensorflow

In [76]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf


BATCH_SIZE = 16

def load_image(image_path):

    image = Image.open(image_path).convert("RGB")  
    image = np.array(image) / 255.0 
    return image

def load_mask(mask_path):

    mask = Image.open(mask_path).convert("L") 
    mask = np.array(mask) / 255.0 
    mask = np.expand_dims(mask, axis=-1) 
    return mask

def parse_filename(filename):

    base_name = os.path.basename(filename)
    count = int(base_name.split("_")[-1].split(".")[0]) 
    return count

def create_dataset(image_dir, mask_dir):

    image_paths = sorted([os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith(".png")])
    mask_paths = sorted([os.path.join(mask_dir, f) for f in os.listdir(mask_dir) if f.endswith(".png")])

    print(f"Number of images: {len(image_paths)}")
    print(f"Number of masks: {len(mask_paths)}")

    if len(image_paths) != len(mask_paths):
        raise ValueError(f"Mismatch between number of images ({len(image_paths)}) and masks ({len(mask_paths)})")

    print("\nFirst 5 image paths:")
    for path in image_paths[:5]:
        print(path)
    print("\nFirst 5 mask paths:")
    for path in mask_paths[:5]:
        print(path)

    images = np.array([load_image(img_path) for img_path in image_paths])
    masks = np.array([load_mask(mask_path) for mask_path in mask_paths])
    counts = np.array([parse_filename(f) for f in image_paths], dtype=np.float32)

    print("\nShapes of loaded data:")
    print(f"Images shape: {images.shape}")
    print(f"Masks shape: {masks.shape}")
    print(f"Counts shape: {counts.shape}")

    dataset = tf.data.Dataset.from_tensor_slices((images, masks, counts))

    dataset = dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    return dataset

image_dir = "Resized_Tiles"
mask_dir = "Generated_Masks"

print(f"Image directory exists: {os.path.exists(image_dir)}")
print(f"Mask directory exists: {os.path.exists(mask_dir)}")

dataset = create_dataset(image_dir, mask_dir)

for images, masks, counts in dataset.take(1):
    print("\nBatch shapes:")
    print("Images shape:", images.shape)
    print("Masks shape:", masks.shape)
    print("Counts:", counts.numpy())

Image directory exists: True
Mask directory exists: True
Number of images: 5
Number of masks: 5

First 5 image paths:
Resized_Tiles/tile_29466_120124_29.png
Resized_Tiles/tile_29466_121142_11.png
Resized_Tiles/tile_31103_115034_21.png
Resized_Tiles/tile_31103_116052_12.png
Resized_Tiles/tile_31103_117070_12.png

First 5 mask paths:
Generated_Masks/tile_29466_120124_mask_29.png
Generated_Masks/tile_29466_121142_mask_11.png
Generated_Masks/tile_31103_115034_mask_21.png
Generated_Masks/tile_31103_116052_mask_12.png
Generated_Masks/tile_31103_117070_mask_12.png

Shapes of loaded data:
Images shape: (5, 512, 512, 3)
Masks shape: (5, 512, 512, 1)
Counts shape: (5,)

Batch shapes:
Images shape: (5, 512, 512, 3)
Masks shape: (5, 512, 512, 1)
Counts: [29. 11. 21. 12. 12.]


In [ ]:
import tensorflow as tf
import tensorflow.keras.layers as layers
import tensorflow.keras.losses as losses
import tensorflow.keras.optimizers as optimizers

IMAGE_SIZE = (512, 512)
BATCH_SIZE = 16

def build_multi_task_model(input_shape=(512, 512, 3)):
    """
    Build a multi-task model with a shared backbone and two heads:
    - Segmentation head (mask prediction)
    - Regression head (count prediction)
    """
    # Shared backbone (e.g., ResNet50)
    backbone = tf.keras.applications.ResNet50(
        include_top=False,
        input_shape=input_shape,
        weights="imagenet"
    )
    backbone.trainable = True  # Fine-tune the backbone


    inputs = layers.Input(shape=input_shape)

    features = backbone(inputs)

    # Segmentation head
    x = layers.Conv2D(512, (3, 3), padding="same", activation="relu")(features)
    x = layers.UpSampling2D((2, 2))(x)  # Upsample to 32x32
    x = layers.Conv2D(256, (3, 3), padding="same", activation="relu")(x)
    x = layers.UpSampling2D((2, 2))(x)  # Upsample to 64x64
    x = layers.Conv2D(128, (3, 3), padding="same", activation="relu")(x)
    x = layers.UpSampling2D((2, 2))(x)  # Upsample to 128x128
    x = layers.Conv2D(64, (3, 3), padding="same", activation="relu")(x)
    x = layers.UpSampling2D((2, 2))(x)  # Upsample to 256x256
    x = layers.Conv2D(32, (3, 3), padding="same", activation="relu")(x)
    x = layers.UpSampling2D((2, 2))(x)  # Upsample to 512x512
    segmentation_output = layers.Conv2D(1, (1, 1), activation="sigmoid", name="segmentation_output")(x)

    # Regression head
    x = layers.GlobalAveragePooling2D()(features)
    x = layers.Dense(128, activation="relu")(x)
    regression_output = layers.Dense(1, name="regression_output")(x)


    model = tf.keras.Model(inputs, [segmentation_output, regression_output])
    return model

def prepare_dataset(dataset):

    def map_fn(image, mask, count):
        return image, (mask, count)
    return dataset.map(map_fn)


prepared_dataset = prepare_dataset(dataset)


model = build_multi_task_model()


model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-4),
    loss={
        "segmentation_output": losses.binary_crossentropy,  # Loss for segmentation
        "regression_output": losses.MSE  # Loss for regression
    },
    loss_weights={
        "segmentation_output": 1.0,  # Weight for segmentation loss
        "regression_output": 1.0  # Weight for regression loss
    },
    metrics={
        "segmentation_output": "accuracy",
        "regression_output": "mae"
    }
)

history = model.fit(
    prepared_dataset,
    epochs=10,
    steps_per_epoch=len(dataset) // BATCH_SIZE,
    validation_data=prepared_dataset,
    validation_steps=len(dataset) // BATCH_SIZE
)

Epoch 1/10


2025-02-03 18:07:51.258946: E tensorflow/core/util/util.cc:131] oneDNN supports DT_INT32 only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.


1/1 ━━━━━━━━━━━━━━━━━━━━ 39s 39s/step - loss: 316.6488 - regression_output_loss: 315.6646 - regression_output_mae: 16.3452 - segmentation_output_accuracy: 0.0897 - segmentation_output_loss: 0.9842 - val_loss: 249.3206 - val_regression_output_loss: 249.0643 - val_regression_output_mae: 14.1367 - val_segmentation_output_accuracy: 0.9918 - val_segmentation_output_loss: 0.2564
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - loss: 243.0906 - regression_output_loss: 242.8461 - regression_output_mae: 14.4730 - segmentation_output_accuracy: 0.9921 - segmentation_output_loss: 0.2446 - val_loss: 220.0009 - val_regression_output_loss: 219.8742 - val_regression_output_mae: 13.0648 - val_segmentation_output_accuracy: 0.9936 - val_segmentation_output_loss: 0.1267
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - loss: 187.7559 - regression_output_loss: 187.6633 - regression_output_mae: 12.9092 - segmentation_output_accuracy: 0.9936 - segmentation_output_loss: 0.0926 - val_loss: 193.6720 - val_regres